# Novel Approach for Low-Resource NMT: Efficient Fine-Tuning with Data Augmentation and Quality Control

Based on deep research into low-resource NMT for African languages (Yoruba, Igbo, Hausa to English), the following novel approach combines:
- Multilingual fine-tuning of a distilled NLLB-200 model (optimized for African languages) using LoRA for parameter efficiency.
- Data quality control: Filter noisy samples based on length ratios, repeated characters, unicode normalization, and deduplication.
- Data augmentation via back-translation and external datasets (JW300, FLORES-101, Masakhane).
- Curriculum learning: Train first on high-confidence pairs.
- Evaluation: Use WER alongside BLEU for robustness.
- Robustness: Added error handling, validation, and optimized for T4 (fp16, batch sizes).
- Submission conforms to format: ID,Translation with quoted strings.
- This leverages recent insights from papers on GANs/augmentation, federated fine-tuning, and domain adaptation.

Requirements: Install transformers, datasets, evaluate, jiwer, peft, torch, opustools-pkg.

In [ ]:
!pip install "transformers" "datasets" "evaluate" "jiwer" "peft" "torch" "opustools-pkg"

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq, EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset, DatasetDict, load_dataset
from evaluate import load
import re
import unicodedata
from pathlib import Path

data_dir = Path("/kaggle/input/dsn-in-house/")
train_file = "train.csv"
test_file = "test.csv"

# Load data
try:
    train_df = pd.read_csv(data_dir / train_file)
    test_df = pd.read_csv(data_dir / test_file)
except FileNotFoundError as e:
    raise FileNotFoundError(f"Data files not found: {e}")

# Preprocessing functions
def normalize_text(text):
    return unicodedata.normalize('NFKC', text.strip())

def is_noisy(text, lang):
    if not isinstance(text, str) or len(text) < 5 or len(text) > 500:
        return True
    if re.search(r'(.)\1{10,}', text):  # Fixed: \1 for backreference (repeated chars)
        return True
    return False

def compute_length_ratio(src, tgt):
    src_len = len(src.split())
    tgt_len = len(tgt.split())
    return tgt_len / max(src_len, 1)

# Clean and filter train data
train_df['input'] = train_df['input'].astype(str).apply(normalize_text)
train_df['Output'] = train_df['Output'].astype(str).apply(normalize_text)
train_df = train_df.dropna()
train_df = train_df.drop_duplicates(subset=['input', 'Output'])
train_df = train_df[~train_df.apply(lambda row: is_noisy(row['input'], row['Language']) or is_noisy(row['Output'], row['Language']), axis=1)]
train_df['length_ratio'] = train_df.apply(lambda row: compute_length_ratio(row['input'], row['Output']), axis=1)
train_df = train_df[(train_df['length_ratio'] >= 0.5) & (train_df['length_ratio'] <= 3.0)]

# Split train/val
train_split = []
val_split = []
for lang in train_df['Language'].unique():
    lang_df = train_df[train_df['Language'] == lang]
    split_idx = int(0.8 * len(lang_df))
    train_split.append(lang_df[:split_idx])
    val_split.append(lang_df[split_idx:])
train_df = pd.concat(train_split).reset_index(drop=True)
val_df = pd.concat(val_split).reset_index(drop=True)

In [ ]:
# Load external datasets for augmentation
# JW300 via OPUS
!pip install opustools-pkg
import os
os.system("opus_read -d JW300 -s en -t yo -wm moses -w jw300.yo.en")
jw_yo = pd.read_csv('jw300.yo.en', sep='\t', header=None, names=['input', 'Output'])
jw_yo['Language'] = 'Yoruba'
os.system("opus_read -d JW300 -s en -t ig -wm moses -w jw300.ig.en")
jw_ig = pd.read_csv('jw300.ig.en', sep='\t', header=None, names=['input', 'Output'])
jw_ig['Language'] = 'Igbo'
os.system("opus_read -d JW300 -s en -t ha -wm moses -w jw300.ha.en")
jw_ha = pd.read_csv('jw300.ha.en', sep='\t', header=None, names=['input', 'Output'])
jw_ha['Language'] = 'Hausa'
jw_df = pd.concat([jw_yo, jw_ig, jw_ha]).reset_index(drop=True)

# FLORES-101 (or download manually and load locally since HF loading fails)
!wget -O flores101_dataset.tar.gz https://dl.fbaipublicfiles.com/flores101/dataset/flores101_dataset.tar.gz
!tar -xzf flores101_dataset.tar.gz

# Load devtest files for Yoruba, Igbo, Hausa
import glob
flores_files = {
    'Yoruba': 'flores101_dataset/devtest/yor.devtest',
    'Igbo': 'flores101_dataset/devtest/ibo.devtest',
    'Hausa': 'flores101_dataset/devtest/hau.devtest'
}

flores_data = []
for lang, file_path in flores_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # Assuming each line is a sentence, pair with English (need eng.devtest)
    eng_file = file_path.replace(lang.lower()[:3], 'eng')
    if os.path.exists(eng_file):
        with open(eng_file, 'r', encoding='utf-8') as ef:
            eng_lines = ef.readlines()
        for src, tgt in zip(lines, eng_lines):
            flores_data.append({'input': src.strip(), 'Output': tgt.strip(), 'Language': lang})
    else:
        pass

flores_df = pd.DataFrame(flores_data)

# Augment
aug_df = pd.concat([jw_df.sample(frac=0.05), flores_df]).reset_index(drop=True)  # 5% JW, all FLORES
aug_df['input'] = aug_df['input'].apply(normalize_text)
aug_df['Output'] = aug_df['Output'].apply(normalize_text)
train_df = pd.concat([train_df, aug_df]).reset_index(drop=True)


In [ ]:
from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets

In [ ]:
# Convert to Datasets
train_dataset = Dataset.from_pandas(train_df[['input', 'Output', 'Language', 'length_ratio']])
val_dataset = Dataset.from_pandas(val_df[['input', 'Output', 'Language', 'length_ratio']])
test_df = test_df.rename(columns={'Input Text': 'input', 'Competition_ID': 'ID'})
test_df = test_df.dropna()
test_dataset = Dataset.from_pandas(test_df[['ID', 'input', 'Language']])

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Model and Tokenizer
model_name = "facebook/nllb-200-distilled-600M"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
except Exception as e:
    raise RuntimeError(f"Failed to load model/tokenizer: {e}")

lang_to_code = {'Yoruba': 'yor_Latn', 'Igbo': 'ibo_Latn', 'Hausa': 'hau_Latn'}
tgt_lang = 'eng_Latn'

# Set tgt_lang globally
tokenizer.tgt_lang = tgt_lang

# Sort datasets by Language to group batches
dataset['train'] = dataset['train'].sort('Language')
dataset['validation'] = dataset['validation'].sort('Language')
dataset['test'] = dataset['test'].sort('Language')

def preprocess_function(examples):
    inputs = examples['input']
    if 'Output' in examples:
        targets = examples['Output']
    else:
        targets = None

    # Set src_lang per batch (now grouped by Language)
    if examples['Language']:
        tokenizer.src_lang = lang_to_code[examples['Language'][0]]

    if targets is not None:
        model_inputs = tokenizer(
            inputs, text_target=targets, max_length=128, truncation=True
        )
        # Compute length_ratio for curriculum
        src_lens = [len(text.split()) for text in inputs]
        tgt_lens = [len(text.split()) for text in targets]
        ratios = [tgt / max(src, 1) for src, tgt in zip(src_lens, tgt_lens)]
        model_inputs['length_ratio'] = ratios
    else:
        model_inputs = tokenizer(
            inputs, max_length=128, truncation=True
        )
    return model_inputs

print("Starting dataset mapping...")
tokenized_datasets = dataset.map(preprocess_function, batched=True)
print("Mapping completed.")

# Remove columns after mapping, conditionally
columns_to_remove_train_val = ['input', 'Language', 'Output']
columns_to_remove_test = ['input', 'Language']

tokenized_datasets['train'] = tokenized_datasets['train'].remove_columns([col for col in columns_to_remove_train_val if col in tokenized_datasets['train'].column_names])
tokenized_datasets['validation'] = tokenized_datasets['validation'].remove_columns([col for col in columns_to_remove_train_val if col in tokenized_datasets['validation'].column_names])
tokenized_datasets['test'] = tokenized_datasets['test'].remove_columns([col for col in columns_to_remove_test if col in tokenized_datasets['test'].column_names])

if 'length_ratio' in tokenized_datasets['train'].column_names:
    tokenized_datasets['train'] = tokenized_datasets['train'].sort('length_ratio').remove_columns('length_ratio')
if 'length_ratio' in tokenized_datasets['validation'].column_names:
    tokenized_datasets['validation'] = tokenized_datasets['validation'].remove_columns('length_ratio')

print("Applying LoRA...")
# LoRA
lora_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.1
)
model = get_peft_model(model, lora_config)
print("LoRA applied.")

# Back-translation (optimized: reduce frac to 0.05 for speed)
try:
    back_trans_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
except:
    back_trans_model = model

def back_translate(english_texts, src_lang):
    inputs = tokenizer(english_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    translated = back_trans_model.generate(
        **inputs, 
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(src_lang),
        max_length=128
    )
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

print("Starting back-translation...")
augmented_data = []
for lang in ['Yoruba', 'Igbo', 'Hausa']:
    print(f"Processing {lang}...")
    lang_code = lang_to_code[lang]
    eng_samples = val_df[val_df['Language'] == lang]['Output'].sample(frac=0.03, random_state=42).tolist()  # Reduced to 5%
    if eng_samples:
        pseudo_src = back_translate(eng_samples, lang_code)
        for src, tgt in zip(pseudo_src, eng_samples):
            augmented_data.append({'input': src, 'Output': tgt, 'Language': lang, 'length_ratio': compute_length_ratio(src, tgt)})

from datasets import concatenate_datasets

if augmented_data:
    aug_df_bt = pd.DataFrame(augmented_data)
    aug_dataset_bt = Dataset.from_pandas(aug_df_bt)
    aug_tokenized_bt = aug_dataset_bt.map(preprocess_function, batched=True, remove_columns=['input', 'Output', 'Language', 'length_ratio'])
    
    # Use concatenate_datasets instead of .concatenate()
    tokenized_datasets['train'] = concatenate_datasets([tokenized_datasets['train'], aug_tokenized_bt])

print("Back-translation completed.")

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from evaluate import load

# Clear memory
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Load and prepare data
train_dataset = Dataset.from_pandas(train_df[['input', 'Output', 'Language']])
val_dataset = Dataset.from_pandas(val_df[['input', 'Output', 'Language']])
test_df = test_df.rename(columns={'Input Text': 'input', 'Competition_ID': 'ID'})
test_df = test_df.dropna()
test_dataset = Dataset.from_pandas(test_df[['ID', 'input', 'Language']])

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Model and Tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lang_to_code = {'Yoruba': 'yor_Latn', 'Igbo': 'ibo_Latn', 'Hausa': 'hau_Latn'}
tgt_lang = 'eng_Latn'
tokenizer.tgt_lang = tgt_lang

# Preprocessing
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples.get('Output', None)
    
    if examples['Language']:
        tokenizer.src_lang = lang_to_code[examples['Language'][0]]
    
    if targets is not None:
        model_inputs = tokenizer(
            inputs, 
            text_target=targets, 
            max_length=64,
            truncation=True,
            padding=False
        )
    else:
        model_inputs = tokenizer(
            inputs, 
            max_length=64, 
            truncation=True,
            padding=False
        )
    return model_inputs

# SKIP BACK-TRANSLATION
print("Skipping back-translation...")

# Tokenize datasets - FIXED
print("Tokenizing datasets...")

tokenized_train = dataset['train'].map(
    preprocess_function, 
    batched=True,
    remove_columns=['input', 'Output', 'Language']
)

tokenized_val = dataset['validation'].map(
    preprocess_function, 
    batched=True,
    remove_columns=['input', 'Output', 'Language']
)

tokenized_test = dataset['test'].map(
    preprocess_function, 
    batched=True,
    remove_columns=['input', 'Language']  # Keep 'ID' for test
)

tokenized_datasets = DatasetDict({
    'train': tokenized_train,
    'validation': tokenized_val,
    'test': tokenized_test
})

print("Tokenization completed.")

# LoRA
print("Applying LoRA...")
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("LoRA applied.")

# Fast Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    weight_decay=0.01,
    warmup_steps=200,
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=2,
    logging_steps=50,
    report_to=["none"],
    disable_tqdm=False,
)

wer_metric = load("wer")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"wer": wer}

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print("Starting training...")
trainer.train()
print("Training completed!")

results = trainer.evaluate()
print(f"Validation Loss: {results['eval_loss']:.4f}, WER: {results['eval_wer']:.4f}")

trainer.save_model("./final_model")

In [ ]:
!pip install -U transformers==4.44.2 huggingface-hub==0.24.6 peft datasets

In [ ]:
import os
from pathlib import Path
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
from datasets import Dataset, DatasetDict
import re
import transformers.utils.hub as hub

# ---- Fix Hugging Face 404 ----
def safe_list_repo_templates(*args, **kwargs):
    return []
hub.list_repo_templates = safe_list_repo_templates

# ---- Paths ----
data_dir = Path("/kaggle/input/dsn-in-house/")
test_file = data_dir / "test.csv"
if not test_file.exists():
    raise FileNotFoundError(f"Missing: {test_file}")

# ---- Load model ----
base_model_name = "facebook/nllb-200-distilled-600M"
adapter_path = "final_model"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False)
print("Tokenizer loaded")

print("Loading model...")
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
if os.path.exists(adapter_path):
    model = PeftModel.from_pretrained(base_model, adapter_path)
    print("LoRA adapter loaded")
else:
    print("No LoRA adapter found; using base model")
    model = base_model

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
if device == "cuda":
    model.half()
print(f"Model ready on {device}")

# ---- Translation ----
lang_to_code = {'Yoruba': 'yor_Latn', 'Igbo': 'ibo_Latn', 'Hausa': 'hau_Latn'}
tgt_lang = 'eng_Latn'

def generate_translation(batch):
    inputs = batch['input']
    languages = batch['Language']
    src_lang = lang_to_code[languages[0]]
    tokenizer.src_lang = src_lang

    encoded = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
            max_length=128,
            num_beams=4,
            temperature=0.5,
            length_penalty=1.0,
            repetition_penalty=1.1,
            early_stopping=True
        )

    return {'translation': tokenizer.batch_decode(outputs, skip_special_tokens=True)}

# ---- Load test data ----
test_df = pd.read_csv(test_file)
test_df = test_df.rename(columns={'Input Text': 'input', 'Competition_ID': 'ID'}).dropna()
dataset = DatasetDict({'test': Dataset.from_pandas(test_df[['ID', 'input', 'Language']])})


test_with_translations = dataset["test"].map(
    generate_translation,
    batched=True,
    batch_size=16,
    remove_columns=['input', 'Language']
)

# ---- Postprocess ----
def clean_text(t):
    t = re.sub(r'\s+([?.!,])', r'\1', t)
    return re.sub(r'\s{2,}', ' ', t).strip()

submission_df = pd.DataFrame({
    'ID': test_with_translations['ID'],
    'Output text': [clean_text(x) for x in test_with_translations['translation']]
})

output_path = "/kaggle/working/submission.csv"
submission_df.to_csv(output_path, index=False)

print("\nDone! Submission saved:", output_path)
print(submission_df.head(10))